In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Path to the zip you just created
zip_path = '/content/drive/MyDrive/augmented_pepper_dataset.zip'
extract_to = '/content/training_data'

# Unzip the data into the local runtime
if not os.path.exists(extract_to):
    print("Extracting augmented dataset...")
    !unzip -q "{zip_path}" -d "{extract_to}"
    print("✅ Extraction complete.")


Extracting augmented dataset...
✅ Extraction complete.


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os


# 2. Define the Path
# Based on your zip structure, the images are located here:
base_dir = '/content/training_data/content/dataset'

# 3. Data Transformations
# We use standard ImageNet normalization for ResNet18
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 4. Load Dataset
# The lambda function ensures we only train on the 'augmented' subfolders
full_dataset = datasets.ImageFolder(
    root=base_dir, 
    transform=data_transforms,
    is_valid_file=lambda x: 'augmented' in x
)

print(f"Detected classes: {full_dataset.classes}")
print(f"Total images found: {len(full_dataset)}")

# 5. Train/Test Split (80/20)
indices = list(range(len(full_dataset)))
train_idx, test_idx = train_test_split(
    indices, 
    train_size=0.8, 
    random_state=42, 
    stratify=full_dataset.targets
)

train_data = Subset(full_dataset, train_idx)
test_data = Subset(full_dataset, test_idx)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 6. Initialize ResNet18
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on: {device}")

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
# Modify the final layer for 2 classes: HEALTHY and UNHEALTHY
model.fc = nn.Linear(model.fc.in_features, 2) 
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 7. Training Loop
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            pbar.set_postfix(loss=loss.item())

        epoch_acc = running_corrects.double() / len(train_data)
        print(f"Training Accuracy: {epoch_acc:.4f}")

    return model

# Start training
model = train_model(model, criterion, optimizer, num_epochs=10)

# 8. Save Final Model to Drive
torch.save(model.state_dict(), '/content/drive/MyDrive/pepper_resnet18_final.pth')
print("✅ Model weights saved to Drive!")

Detected classes: ['HEALTHY', 'UNHEALTHY']
Total images found: 47952
Training on: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 180MB/s]
Epoch 1/10: 100%|██████████| 1199/1199 [05:12<00:00,  3.84it/s, loss=0.0216]  


Training Accuracy: 0.9954


Epoch 2/10: 100%|██████████| 1199/1199 [04:48<00:00,  4.16it/s, loss=3.78e-5] 


Training Accuracy: 0.9979


...

Training Accuracy: 1.0000


Epoch 9/10: 100%|██████████| 1199/1199 [04:42<00:00,  4.25it/s, loss=0.00813] 


Training Accuracy: 0.9989


Epoch 10/10: 100%|██████████| 1199/1199 [04:40<00:00,  4.27it/s, loss=6.94e-5] 


Training Accuracy: 0.9996
✅ Model weights saved to Drive!
